In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import datetime
import math, time
import itertools
from sklearn import preprocessing
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from math import sqrt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [2]:
def get_stock_data(stock_name, normalized=0):
    
    url="http://www.google.com/finance/historical?q="+stock_name+"&startdate=Aug+03%2C+2015&enddate=Aug+02%2C+2017&num=30&ei=d7d4WbrtDYq1swH0mp-4CA&output=csv" 
    print(url)
    col_names = ['Date','Open','High','Low','Close','Volume']
    stock = pd.read_csv(url, header=0, names=col_names) 
    df = pd.DataFrame(stock)

    df["Volume"] = df["Volume"] / 10000
    df.drop(df.columns[[0,3,5]], axis=1, inplace=True) 
    return df

In [3]:
stock_name = 'GOOGL'
df = get_stock_data(stock_name,0)
df.head()

http://www.google.com/finance/historical?q=GOOGL&startdate=Aug+03%2C+2015&enddate=Aug+02%2C+2017&num=30&ei=d7d4WbrtDYq1swH0mp-4CA&output=csv


,Open,High,Close
0,948.37,949.10,947.64
1,947.81,954.49,946.56
2,960.00,961.19,945.50
3,947.99,961.79,958.33
4,969.18,969.52,952.51


In [4]:
# Normalization

df['High'] = df['High'] / 1000
df['Open'] = df['Open'] / 1000
df['Close'] = df['Close'] / 1000
df.head(20)

,Open,High,Close
0,0.94837,0.94910,0.94764
1,0.94781,0.95449,0.94656
2,0.96000,0.96119,0.94550
3,0.94799,0.96179,0.95833
4,0.96918,0.96952,0.95251
5,0.97278,0.97395,0.96531
6,0.97070,0.97673,0.96903
7,0.99410,1.00619,0.99831
8,0.98900,0.99511,0.99384
9,0.99700,0.99868,0.99219


In [5]:
def load_data(df, seq_len):
    amount_of_features = len(df.columns)
    data = df.as_matrix()
    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])

    result = np.array(result)
    row = 1.0 * result.shape[0]
    train = result[:int(row), :]
    x_train = train[:-3, :-1]
    y_train = train[3:, -1][:,-1]
    x_test = result[int(row):-3, :-1]
    y_test = result[int(row)+3:, -1][:,-1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features))  

    return [x_train, y_train, x_test, y_test]

In [6]:
window = 10
X_train, y_train, X_test, y_test = load_data(df[::-1], window)
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)
print(X_train[:-1][:,-1][:10])
print(y_train[:-1][:10])

X_train (491, 10, 3)
y_train (491,)
X_test (0, 10, 3)
y_test (0,)
[[ 0.68404  0.69226  0.68937]
 [ 0.68804  0.69474  0.69411]
 [ 0.69107  0.69576  0.68873]
 [ 0.69011  0.7002   0.69404]
 [ 0.6891   0.69616  0.67948]
 [ 0.67104  0.67289  0.64403]
 [ 0.6006   0.64433  0.61811]
 [ 0.64647  0.647    0.61247]
 [ 0.6431   0.66248  0.65974]
 [ 0.67199  0.67398  0.66796]]
[ 0.67948  0.64403  0.61811  0.61247  0.65974  0.66796  0.65969  0.64782
  0.62956  0.64491]


In [7]:
def build_model2(layers):
    d = 0.2
    model = Sequential()
    model.add(LSTM(512, input_shape=(layers[1], layers[0]), return_sequences=True))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(d))
    
    model.add(LSTM(128, input_shape=(layers[1], layers[0]), return_sequences=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(d))
    
    model.add(Dense(32,init='uniform',activation='relu'))        
    model.add(Dense(1,init='uniform',activation='relu'))
    model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
    return model

amount_of_features = len(df.columns)
# model = build_model([amount_of_features,lag,1])
model = build_model2([amount_of_features,window,1])

In [8]:
model.fit(
    X_train,
    y_train,
    batch_size=256,
    nb_epoch=200,
#     validation_split=0.2,
    verbose=1)

Epoch 1/200
491/491 [==============================] - 3s - loss: 0.6198 - acc: 0.0000e+00     
Epoch 2/200
491/491 [==============================] - 1s - loss: 0.5564 - acc: 0.0000e+00     
Epoch 3/200
491/491 [==============================] - 1s - loss: 0.5047 - acc: 0.0000e+00     
Epoch 4/200
491/491 [==============================] - 1s - loss: 0.4527 - acc: 0.0000e+00     
Epoch 5/200
491/491 [==============================] - 1s - loss: 0.4002 - acc: 0.0000e+00     
Epoch 6/200
491/491 [==============================] - 1s - loss: 0.3512 - acc: 0.0000e+00     
Epoch 7/200
491/491 [==============================] - 1s - loss: 0.2993 - acc: 0.0000e+00     
Epoch 8/200
491/491 [==============================] - 1s - loss: 0.2554 - acc: 0.0000e+00     
Epoch 9/200
491/491 [==============================] - 1s - loss: 0.2199 - acc: 0.0000e+00     
Epoch 10/200
491/491 [==============================] - 1s - loss: 0.1853 - acc: 0.0000e+00     
Epoch 11/200
491/491 [=================

In [9]:
trainScore = model.evaluate(X_train, y_train, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))
print("%s: %.2f%%\n" % (model.metrics_names[1], trainScore[1]*100))

# testScore = model.evaluate(X_test, y_test, verbose=0)
# print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore[0], math.sqrt(testScore[0])))
# print("%s: %.2f%%" % (model.metrics_names[1], testScore[1]*100))

Train Score: 0.00 MSE (0.06 RMSE)
acc: 0.00%



In [10]:
my_test = np.array([np.array(df[0:window][::-1])])
print(my_test)
print(model.predict(my_test, batch_size=1, verbose=1))

[[[ 0.997    0.99868  0.99219]
  [ 0.989    0.99511  0.99384]
  [ 0.9941   1.00619  0.99831]
  [ 0.9707   0.97673  0.96903]
  [ 0.97278  0.97395  0.96531]
  [ 0.96918  0.96952  0.95251]
  [ 0.94799  0.96179  0.95833]
  [ 0.96     0.96119  0.9455 ]
  [ 0.94781  0.95449  0.94656]
  [ 0.94837  0.9491   0.94764]]]
1/1 [==============================] - 0s
[[ 0.88795871]]
